# Estimating SB ages using an MC approach 
---
McDermid et al. (2015) state that they use a Monte Carlo approach to estimate 1$\sigma$ errors on their mass-weighted age estimates. To do this they use ppxf *without* regularisation, but to estimate the mass-weighted age they do use regularisation.

Here, we are going to try using an MC approach, but without regularisation, to see whether we can accurately recover the mass-weighted age of the young component in the stellar population. If this works, then we can do away with regulariation, as it is very computationally expensive. 

To summarise, we will
1. Define the "truth" SFH and generate the corresponding spectrum.
2. In each MC iteration,
    1. add *additional* random noise to the spectrum.
    2. run ppxf. 
    3. compute the mass-weighted age of the young component. 
3. From the ensemble of mass-weighted age measurements, compute the mean and standard deviation. Do these rouhgly correspond to the input SFH?

Need to
* Incorporate multithreading to run multiple ppxf instances simultaneously.

In [1]:
%matplotlib widget

In [80]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [152]:
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
import multiprocessing

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.ppxf_plot import plot_sfh_mass_weighted

import matplotlib.pyplot as plt
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

In [66]:
###########################################################################
# Settings
###########################################################################
isochrones = "Padova"
SNR = 100
sigma_star_kms = 250
z = 0.01

In [122]:
###########################################################################
# Generate the input SFH
###########################################################################
# Load the stellar templates so we can get the age & metallicity dimensions
_, _, metallicities, ages = load_ssp_templates(isochrones)
N_ages = len(ages)
N_metallicities = len(metallicities)

# Simple Gaussian SFH
xx, yy = np.meshgrid(range(N_ages), range(N_metallicities))
x0 = 10
y0 = 1.5
sigma_x = 1
sigma_y = 0.1
sfh_young = np.exp(- (xx - x0)**2 / (2 * sigma_x**2)) *\
               np.exp(- (yy - y0)**2 / (2 * sigma_y**2))
sfh_young /= np.nansum(sfh_young)
sfh_mw_young = sfh_young * 1e8

x0 = 60
y0 = 0
sigma_x = 1
sigma_y = 0.5
sfh_old = np.exp(- (xx - x0)**2 / (2 * sigma_x**2)) *\
             np.exp(- (yy - y0)**2 / (2 * sigma_y**2))
sfh_old /= np.nansum(sfh_old)
sfh_mw_old = sfh_old * 1e10

# Add the young & old components
sfh_mw_original = sfh_mw_old + sfh_mw_young

# # Instead of a Gaussian SFH, try with a "delta function" SFH (i.e., 1 old & 1 young template)
# sfh_mw_original = np.zeros((N_metallicities, N_ages))
# sfh_mw_original[2, 4] = 1e7
# sfh_mw_original[1, -3] = 1e10

sfh_mw_1D_original = np.nansum(sfh_mw_original, axis=0)


In [119]:
###########################################################################
# Function for computing the mass-weighted age 
###########################################################################
def calculate_mw_age(sfh_mw, age_thresh, ages):
    # Sum the SFH over the metallicity dimension to get the 1D SFH
    sfh_mw_1D = np.nansum(sfh_mw, axis=0) if sfh_mw.ndim > 1 else sfh_mw

    # Find the index of the threshold age in the template age array
    age_thresh_idx = np.nanargmin(np.abs(ages - age_thresh))
    
    # Compute the mass-weighted age 
    log_age_mw = np.nansum(sfh_mw_1D[:age_thresh_idx] * np.log10(ages[:age_thresh_idx])) / np.nansum(sfh_mw_1D[:age_thresh_idx])
    
    # Compute the corresponding index in the array (useful for plotting)
    log_age_mw_idx = np.nanargmin(np.abs(np.log10(ages) - log_age_mw))
    
    return log_age_mw, log_age_mw_idx
    

In [126]:
# Test that it works 
log_age_mw, log_age_mw_idx = calculate_mw_age(sfh_mw_original, age_thresh=1e9, ages=ages)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
ax.step(range(N_ages), sfh_mw_1D_original, color="black", where="mid")
ax.axvline(log_age_mw_idx, color="red")
ax.set_ylim([1, None])
ax.set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## MONTE CARLO TESTING
---
Each iteration, use the same spectrum, but with random noise added to it. 

In [123]:
###########################################################################
# Generate the spectrum
###########################################################################
spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [161]:
list(np.random.randint(low=0, high=100 * niters, size=niters))

[1612,
 1726,
 910,
 857,
 296,
 926,
 1664,
 625,
 247,
 186,
 231,
 1864,
 1928,
 1959,
 161,
 1569,
 182,
 32,
 1180,
 343]

In [162]:
###########################################################################
# Run ppxf WITHOUT regularisation, using a MC approach to estimate the SFH
###########################################################################
# Helper function for multiprocessing
def ppxf_helper(seed):
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_original_err)
    spec = spec_original + noise
    
    # print(noise[0])  # For checking whether the random noise is being reseeded each iteration
    
    spec[0] = -9999
    spec[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp
     
# Input arguments
niters = 20
nthreads = 20
args_list = list(np.random.randint(low=0, high=100 * niters, size=niters))

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Elapsed time in ppxf: {time() - t:.2f} s")


Running ppxf on 20 threads...


6.983376622581931e+36
-9.051531587922739e+36
7.282262118343539e+35
5.097978099931293e+36
-4.449573959205587e+36
-3.0942375085078365e+36
-1.0490392625277806e+37
-4.468190636624613e+36
-8.064298445141361e+35
-1.9662509447246135e+37
4.897663750920843e+37
1.3929907197868998e+37
-2.8227967444947747e+37
-9.148598993951686e+36
2.5478103999477496e+37
-1.4954613696031168e+37
6.776975201355106e+36
-9.051531587922739e+36
2.2085907857811065e+34
-1.3018060353179432e+37

Elapsed time in ppxf: 44.19 s


In [163]:
###########################################################################
# COMPARE THE INPUT AND OUTPUT
###########################################################################
sfh_fit_mw_list = []
sfh_fit_lw_list = []
sfh_fit_mw_1D_list = []
sfh_fit_lw_1D_list = []

for pp in pp_list:
    sfh_fit_mw_list.append(pp.weights_mass_weighted)
    sfh_fit_lw_list.append(pp.weights_light_weighted)
    sfh_fit_mw_1D_list.append(np.nansum(pp.weights_mass_weighted, axis=0))
    sfh_fit_lw_1D_list.append(np.nansum(pp.weights_light_weighted, axis=0))
    
# Compute the mean SFH 
sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)

# Compute the mass-weighted 

# Plot the mass-weighted weights, summed over the metallicity dimension
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
fig.subplots_adjust(hspace=0.3)
axs = axs.flat
axs[0].set_title("Mass-weighted template weights (different random realisations)")
for ax in axs:
    ax.step(range(N_ages), sfh_mw_1D_original, color="black", where="mid", label="Input SFH")
    for jj in range(niters):
        ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="red", alpha=0.1, where="mid", label="ppxf fits" if jj == 0 else None)
    ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="blue", where="mid", label="Mean ppxf fit")
    
    ax.legend()
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylim([1, None])
    ax.set_ylabel("Template weight")
axs[1].set_yscale("log")

# Plot the light-weighted weights, summed over the metallicity dimension
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
fig.subplots_adjust(hspace=0.3)
axs = axs.flat
axs[0].set_title("Light-weighted template weights (different random realisations)")
for ax in axs:
    for jj in range(niters):
        ax.step(range(N_ages), sfh_fit_lw_1D_list[jj], color="red", alpha=0.1, where="mid", label="ppxf fits" if jj == 0 else None)
    ax.step(range(N_ages), sfh_fit_lw_1D_mean, color="blue", where="mid", label="Mean ppxf fit")
    
    ax.legend()
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylabel("Template weight")
axs[1].set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Method 1: run ppxf ~100 times *without* regularisation
---
#### Remarks
* `ppxf` does NOT yield exactly the same result each time it is run, even if `regul` is fixed! In this scenario, the S/N on the input doesn't seem to make much difference - i.e., even if the S/N is very high, the output still varies. **However**, `ppxf` is still able to correctly identify the *peaks* in the SFH.  

In [ ]:
###########################################################################
# Run ppxf WITHOUT regularisation, with the SAME input spectrum & noise each time.
###########################################################################
# Generate the spectrum
spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_original,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)  

spec_original[0] = -9999
spec_original[-1] = -9999

# Run ppxf
pp_list = []
for ii in tqdm(range(100)):
    pp = run_ppxf(spec=spec_original, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    pp_list.append(pp)

In [82]:
###########################################################################
# COMPARE THE INPUT AND OUTPUT
###########################################################################
sfh_fit_mw_list = []
sfh_fit_lw_list = []
sfh_fit_mw_1D_list = []
sfh_fit_lw_1D_list = []

for pp in pp_list:
    sfh_fit_mw_list.append(pp.weights_mass_weighted)
    sfh_fit_lw_list.append(pp.weights_light_weighted)
    sfh_fit_mw_1D_list.append(np.nansum(pp.weights_mass_weighted, axis=0))
    sfh_fit_lw_1D_list.append(np.nansum(pp.weights_light_weighted, axis=0))

# Plot the mass-weighted weights, summed over the metallicity dimension
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
fig.subplots_adjust(hspace=0.3)
axs = axs.flat
axs[0].set_title("Mass-weighted template weights (identical inputs)")
for ax in axs:
    ax.step(range(N_ages), sfh_mw_1D_original,
            color="black", where="mid", label="Input SFH")
    for jj in range(ii + 1):
        ax.step(range(N_ages), sfh_fit_mw_1D_list[jj],
                color="red", alpha=0.1, where="mid", label="ppxf fits" if jj == 0 else None)
    ax.legend()
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylim([1, None])
    ax.set_ylabel("Template weight")
axs[1].set_yscale("log")

# Plot the light-weighted weights, summed over the metallicity dimension
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
fig.subplots_adjust(hspace=0.3)
axs = axs.flat
axs[0].set_title("Light-weighted template weights (identical inputs)")
for ax in axs:
    for jj in range(ii + 1):
        ax.step(range(N_ages), sfh_fit_lw_1D_list[jj],
                    color="red", alpha=0.1, where="mid", label="ppxf fits" if jj == 0 else None)
    ax.legend()
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylabel("Template weight")
axs[1].set_yscale("log")

----------------------------------------------------
Elapsed time in PPXF: 4.59 s
----------------------------------------------------
Elapsed time in PPXF: 3.99 s
----------------------------------------------------
Elapsed time in PPXF: 3.34 s
----------------------------------------------------
Elapsed time in PPXF: 3.49 s
----------------------------------------------------
Elapsed time in PPXF: 3.80 s
----------------------------------------------------
Elapsed time in PPXF: 3.73 s
----------------------------------------------------
Elapsed time in PPXF: 3.71 s
----------------------------------------------------
Elapsed time in PPXF: 3.39 s
----------------------------------------------------
Elapsed time in PPXF: 3.80 s
----------------------------------------------------
Elapsed time in PPXF: 3.77 s
----------------------------------------------------
Elapsed time in PPXF: 3.51 s
----------------------------------------------------
Elapsed time in PPXF: 3.69 s
----------------

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
###########################################################################
# Run ppxf WITHOUT regularisation, with a new random reasliation of the 
# input spectrum & noise each time.
###########################################################################
pp_list = []
for ii in tqdm(range(100)):
    # Generate the spectrum
    spec_original, spec_original_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_original,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)  

    spec_original[0] = -9999
    spec_original[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_original, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1,
                  regularisation_method="none", 
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    pp_list.append(pp)

100%|██████████| 100/100 [09:25<00:00,  5.65s/it]


In [91]:
###########################################################################
# COMPARE THE INPUT AND OUTPUT
###########################################################################
sfh_fit_mw_list = []
sfh_fit_lw_list = []
sfh_fit_mw_1D_list = []
sfh_fit_lw_1D_list = []

for pp in pp_list:
    sfh_fit_mw_list.append(pp.weights_mass_weighted)
    sfh_fit_lw_list.append(pp.weights_light_weighted)
    sfh_fit_mw_1D_list.append(np.nansum(pp.weights_mass_weighted, axis=0))
    sfh_fit_lw_1D_list.append(np.nansum(pp.weights_light_weighted, axis=0))
    
# Compute the mean SFH 
sfh_fit_mw_mean = np.nansum(np.array(sfh_fit_mw_list), axis=0) / len(sfh_fit_mw_list)
sfh_fit_lw_mean = np.nansum(np.array(sfh_fit_lw_list), axis=0) / len(sfh_fit_lw_list)
sfh_fit_mw_1D_mean = np.nansum(sfh_fit_mw_mean, axis=0)
sfh_fit_lw_1D_mean = np.nansum(sfh_fit_lw_mean, axis=0)

# Plot the mass-weighted weights, summed over the metallicity dimension
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
fig.subplots_adjust(hspace=0.3)
axs = axs.flat
axs[0].set_title("Mass-weighted template weights (different random realisations)")
for ax in axs:
    ax.step(range(N_ages), sfh_mw_1D_original, color="black", where="mid", label="Input SFH")
    for jj in range(ii + 1):
        ax.step(range(N_ages), sfh_fit_mw_1D_list[jj], color="red", alpha=0.1, where="mid", label="ppxf fits" if jj == 0 else None)
    ax.step(range(N_ages), sfh_fit_mw_1D_mean, color="blue", where="mid", label="Mean ppxf fit")
    
    ax.legend()
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylim([1, None])
    ax.set_ylabel("Template weight")
axs[1].set_yscale("log")

# Plot the light-weighted weights, summed over the metallicity dimension
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(10, 6))
fig.subplots_adjust(hspace=0.3)
axs = axs.flat
axs[0].set_title("Light-weighted template weights (different random realisations)")
for ax in axs:
    for jj in range(ii + 1):
        ax.step(range(N_ages), sfh_fit_lw_1D_list[jj], color="red", alpha=0.1, where="mid", label="ppxf fits" if jj == 0 else None)
    ax.step(range(N_ages), sfh_fit_lw_1D_mean, color="blue", where="mid", label="Mean ppxf fit")
    
    ax.legend()
    ax.set_xticks(range(N_ages))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical", fontsize="x-small")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylabel("Template weight")
axs[1].set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
plt.close("all")

### Remarks

In [18]:
###########################################################################
# Fit with ppxf
###########################################################################
t = time()
pp1 = run_ppxf(spec=spec_original, spec_err=spec_original_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1,
              auto_adjust_regul=True,
              isochrones="Padova",
              fit_gas=False, tie_balmer=True,
              delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
              plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

sfh_lw_pp1 = pp1.weights_light_weighted
sfh_mw_pp1 = pp1.weights_mass_weighted

Median SNR = 99999.9873
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.04 s
----------------------------------------------------
Iteration 1: Scaling noise by 1932.8343...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 58.24 s
Iteration 1: optimal regul = 500.00; Δm = 1.21455e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 2.462
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 47.67 s
Iteration 2: optimal regul = 500.00; Δm = 0; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 2.462
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 49.42 s
Iteration 3: optimal regul = 520.00; Δm = 8.2726e+07; Δregul = 20.00 (Δregul_min = 1.00); Δχ (goal) - Δ

/home/u5708159/python/Modules/ppxftests/run_ppxf.py:425: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(15, 6.5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total time in run_ppxf: 161.25 seconds


In [21]:
###########################################################################
# Re-create the input spectrum from ppxf 
###########################################################################
spec_pp1, spec_pp1_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_pp1,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)  

spec_pp1[0] = -9999
spec_pp1[-1] = -9999

# Compare 
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5))
ax.plot(lambda_vals_A, spec_original, color="black", label="spec_original")
ax.plot(lambda_vals_A, spec_pp1, color="red", label="spec_pp1")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  del sys.path[0]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
###########################################################################
# Re-run ppxf to check that it correctly fits the input SFH
###########################################################################
t = time()
pp2 = run_ppxf(spec=spec_pp1, spec_err=spec_pp1_err, lambda_vals_A=lambda_vals_A,
               z=z, ngascomponents=1,
               auto_adjust_regul=True,
               isochrones="Padova",
               fit_gas=False, tie_balmer=True,
               delta_regul_min=1, regul_max=5e4, delta_delta_chi2_min=1,
               plotit=False, savefigs=False, interactive_mode=False)
print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

sfh_lw_pp2 = pp2.weights_light_weighted
sfh_mw_pp2 = pp2.weights_mass_weighted

Median SNR = 100000.0171
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.31 s
----------------------------------------------------
Iteration 1: Scaling noise by 1947.7944...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 53.34 s
Iteration 1: optimal regul = 3500.00; Δm = 1.03137e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 4.004
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 59.64 s
Iteration 2: optimal regul = 3300.00; Δm = 9.46714e+07; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.444
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.4439487130693891; using 3300.00 to produce the best fit


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Total time in run_ppxf: 119.13 seconds


In [ ]:
###########################################################################
# CHECK: compare the input & output SFHs 
###########################################################################
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20, 11))
fig.subplots_adjust(hspace=0.3)
# Mass-weighted 
for ax in [axs[0][0], axs[1][0]]:
    ax.step(range(len(sfh_mw_original[0])), sfh_mw_original[0], color="black", where="mid", label="Original input SFH")
    ax.step(range(len(sfh_mw_pp1[0])), sfh_mw_pp1[0], color="red", alpha=0.1, where="mid", label="ppxf fit 1")
    ax.step(range(len(sfh_mw_pp2[0])), sfh_mw_pp2[0], color="green", alpha=0.1, where="mid", label="ppxf fit 2")
    ax.legend()
    ax.set_xticks(range(len(ages)))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylim([1, None])
    ax.set_ylabel("Template weight (mass-weighted)")
axs[1][0].set_yscale("log")

# light-weighted 
for ax in [axs[0][1], axs[1][1]]:
    ax.step(range(len(sfh_lw_pp1[0])), sfh_lw_pp1[0], color="red", alpha=0.1, where="mid", label="ppxf fit 1")
    ax.step(range(len(sfh_lw_pp2[0])), sfh_lw_pp2[0], color="green", alpha=0.1, where="mid", label="ppxf fit 2")
    ax.legend()
    ax.set_xticks(range(len(ages)))
    ax.set_xlabel("Age (Myr)")
    ax.set_xticklabels(["{:}".format(age / 1e6) for age in ages], rotation="vertical")
    ax.autoscale(axis="x", enable=True, tight=True)
    ax.set_ylabel("Template weight (light-weighted)")
axs[1][1].set_yscale("log")

In [ ]:
###########################################################################
# Save the mass weights to a FITS file so that we can load it 
###########################################################################
# Create a .fits file 
header = fits.Header()
header["NAGES"] = N_ages
header["NMET"] = N_metallicities
header["ISCHRN"] = isochrones
header["MTOT"] = np.nansum(sfh_mw_pp2)
header["LOGMTOT"] = np.log10(np.nansum(sfh_mw_pp2))
hdu_primary = fits.PrimaryHDU(header=header)

hdu_sfh_mw = fits.ImageHDU(sfh_mw_pp2, name="SFH_MW")
hdu_sfh_lw = fits.ImageHDU(sfh_lw_pp2, name="SFH_LW")
hdulist = fits.HDUList([hdu_primary, hdu_sfh_mw, hdu_sfh_lw])

hdulist.writeto("SFHs/sfh_mw_old+young.fits", overwrite=True)